In [1]:
import os
import numpy
os.chdir("utils")
from burgers_utils import *
from mlp_network import shared_model_func
from plotting import list_tensor_to_list
import pandas as pd
import time

In [2]:
file_path = '../data/burgers_shock.mat'

In [3]:
# loading data from the burgers_shock.mat
# usol in the paper's script has been reffered as Exact
x, t, usol = load_data(file_path)

In [4]:
# making a spatial-temporal grid
X, T = mesh_grid(x, t)

In [5]:
# Creating test data set
# X_test is reffered as X_star
# y_test is reffered as u_star
X_test, y_test = test_data(X, T, usol)

In [6]:
# Domain bounds 
lb, ub = domain_bounds(X_test)

In [7]:
# Number of collocation points i.e. number of training points for f
N_f = 10000
# Number of initial and boudary conditions to train the model for u
N_u = 100

In [8]:
### In Physics-informed neural networks, we are predicting a latent variable u (x,t) and structure of 
### the pde f(u,x, t). variables with "u_train" in them corresponds to the training example required for u 
### and "f_train" corresponds to the training example required for f
### We don't have "y_f_train" because it's value is always zero and we specified it in the loss function

In [9]:
# In the the paper they are reffered as X_u_train, X_f_train and u_train
x_f, x_u, y_u = training_data(X, T, usol, lb, ub, N_f)

In [10]:
# From the entire initial and boundary conditions we will select N_u number of x,t,u for the training
idx = np.random.choice(x_u.shape[0], N_u, replace=False)
X_u_train = x_u[idx, :]
y_u_train = y_u[idx, :]
# Stacking collocation points and IC and BC condtions
# this will be used for imposing the structure of the Partial Differential Equation
X_f_train = np.vstack([x_f, X_u_train])

In [11]:
# Specifying the number of layers in the neural network shared between u and f
layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]
# Specifying the coefficient of viscosity
nu = 0.01/np.pi

## Hyper parameter search

In [12]:
## Here, we are investigating different optimizers, learning rate, depth and widht of the network.
## We are not changing the activation functions (just keeping the same as specified in the paper).

In [13]:
optim = ['Adam', 'RMSprop', 'SGD']
lr_rate = []
# Trying to perform random grid search by getting random learning rate
std_rate = [1e-3, 5*1e-2, 1e-2, 1e-1]
for i in range(4):
    a = np.round(np.random.uniform(0, 1, size=1)*std_rate[i], 4)
    lr_rate.append(a[0])
print("Learning rates used for hyparameters optimization are: {}\n".format(lr_rate))
depth = [4, 6, 8, 10] # Number of layers
width = [10, 20, 30] # Number of neurons each layer
print("Total trials for the hyperparameters search are: {}\n".format(len(optim) * len(lr_rate) * len(depth) * len(width)))
print()  


Learning rates used for hyparameters optimization are: [0.0009, 0.0397, 0.0098, 0.0248]

Total trials for the hyperparameters search are: 144




In [14]:
# @tf.function
def model_build_compile(x_f, t_f, x_u, t_u,u, shared_model, loss_function, optimizer):
    '''
    model_build_compile: function will build the model, compile with loss function and optimizer,
    calculate the gradients for weights and updating them automatically
    Arguemnts:
    x_f-- tensorflow batch dataset of shape (None, 1) [Pass x training data for collocation f]
    t_f-- tensorflow batch dataset of shape (None, 1) [Pass t training data for collocation f]
    x_u-- a numpy ndarray of shape (None, 1) [Pass x training data for latent variable u]
    t_u-- a numpy ndarray of shape (None, 1) [Pass t training data for latent variable u]
    u-- a numpy ndarray of shape (None, 1) [Pass training labels for latent variable u]
    shared_model-- the keras model which is shared between u and f network
    loss_function-- the keras loss function (for this project we use MSE)
    optimizer-- the keras optimizer function (please specify the name, learning rate)

    Returns
    loss_u-- a mean squared tensorflow loss calcuated for variable u
    loss_f-- a mean squared tensorflow loss calcuated for collocation f
    loss_combine-- combinted loss of variable u and collocation f loss_combine = loss_u + loss_f
    '''
    # We have 2 different loss functions here
    # loss_1 corresponds to loss in u
    # loss_2 corresponds to loss in f
    # Typicall plan, i.e. build model, compile model and fit model were not working in our case
    # We referred this link for training the model
    # Reference: https://www.tensorflow.org/guide/function
    # Reference: https://keras.io/api/optimizers/#learning-rate-decay--scheduling
    # Reference: https://colab.research.google.com/drive/1lo7Kf8zTb-DF_MjkO8Y07sYELnX3BNUR#scrollTo=GkimJNtepkKi
    # Reference: https://pgaleone.eu/tensorflow/tf.function/2019/03/21/dissecting-tf-function-part-1/
    # Reference: https://github.com/helonayala/pinn/blob/main/01_DeepVIV_sec21.ipynb
    # This apprach i.e. optmizers.minimize(cost/loss) is also mentioned in the original paper

    # In the paper, they specify mean squared error for variable u and f

    # learning schedule for the optimizer

    # In the original paper, they used scipy "L-BFGS" optimizer but we are using Adam
    # L-BFGS is not available in tf. We can get it via tensorflow probability distribution but it's available
    # for tf>=2.3
#     if opt == 'Adam':
#         optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_schedule)

    # for the pde structure we need some derivatives of u, so we are using gradient tape to do it for us
    # Reference: https://www.tensorflow.org/guide/autodiff
    with tf.GradientTape() as tape_out:
        # Trainign for variable u i.e IC and BC
        u_pred = shared_model([x_u, t_u], training=True)
        loss_u = loss_function(u, u_pred)

        # Trainign for variable f i.e collocation points
        with tf.GradientTape(persistent=True) as tape_in:
            tape_in.watch(x_f)
            tape_in.watch(t_f)

            u_f = shared_model([x_f, t_f], training=True)
            # Making a PDE net using automatic differentiation technique
            # f = u_t + u * u_x - nu * u_xx
            # Calling gradient tape here, because want to trace everything
            # term1 ()
            u_x = tape_in.gradient(u_f, x_f)
            # term 2
            u_t = tape_in.gradient(u_f, t_f)
        #term 3
        u_xx = tape_in.gradient(u_x, x_f)

        f_pred = u_t + u_f * u_x - 0.01/np.pi*u_xx
        # Value of f_pred is zero
        loss_f = loss_function(f_pred, tf.convert_to_tensor(0, dtype=tf.float64))

        # Once we calculate loss for each variable and now combine them
        # MSE(u, u_pred) + MSE(0, f_pred)
        loss_combine = loss_u + loss_f

    # As given on the webpages
    grads = tape_out.gradient(loss_combine, shared_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, shared_model.trainable_weights))

    del loss_function
    del optimizer
    return loss_u, loss_f, loss_combine



In [15]:
def best_n_model(track_dict, best=20):
    new_dict = {}
    error_track = []
    best_model = []
    for k, v in track_dict.items():
        for k1,v1 in v.items():
            if k1 == 'error':
                error_track.append(v1)
                best_model.append(int(k))
    error_track_1, best_model_1 = zip(*sorted(zip(error_track, best_model)))
    for model in best_model_1[0:best]:
#         print("************************")
#         print("Best models are: \n")
#         print(track_dict['{}'.format(model)])
#         print("************************\n")
        new_dict['{}'.format(model)] = track_dict['{}'.format(model)]
    return new_dict 

In [16]:
# For different learning rates
def param_tuning(optim, lr_rate, depth, width, loss_function, train_dataset):
    track_dict = {}
    inp_num = 2
    out_num = 1
    klm = 0
    for opt in tqdm(optim):
        for lr in lr_rate:
            # For different number of layers in a network
            for dep in depth:
                # For different number of units in a layer
                for wid in width:
                    small_dict = {}
                    # Creating a list of size number of layers + 2
                    layers = [None]*(dep+2)
                    # Assigning size of input = 2
                    layers[0] = inp_num
                    # Assigning size of output = 1
                    layers[-1] = out_num
                    # Assigning number of neurons in hidden layers
                    layers[1:-1] = wid*np.ones(dep, dtype=int)
                    # Creating a model
                    shared_model = shared_model_func(layers=layers, lb=lb, ub = ub, norm = True)
                    # Keeping the same loss function as specified in the paper
                    # Keeping the lear_rate)sched =. False
                    optimizer = choose_optimizer(lr=lr, ds=100, er=0.96, opt=opt, lear_rate_sched=False)
                    # Fixing maximum number of epochs 
                    max_Iter = 1000
                    loss_model = []
                    start = time.time()
                    for epoch in range(max_Iter):
                        for (X_f_train, t_f_train) in train_dataset:   
                            # Passing the entire data in a single batch (That's how they did in the original paper)
                            _, _, loss_combine = model_build_compile(x_f=X_f_train, t_f=t_f_train, 
                                                               x_u = X_u_train[:,0:1], t_u = X_u_train[:,1:2],
                                                               u = y_u_train, shared_model=shared_model, 
                                                               loss_function=loss_function,
                                                               optimizer=optimizer)

                            loss_model.append(loss_combine)
                    y_hat = predict(shared_model, X_test)
                    small_dict['optimizer'] = opt
                    small_dict['learning_rate'] = lr
                    small_dict['number of layer'] = dep
                    small_dict['number of neurons'] = wid
                    small_dict['loss model'] = list_tensor_to_list(loss_model)
                    small_dict['error'] = error_loss(y_test, y_hat)
                    track_dict['{}'.format(klm)] = small_dict
                    end = time.time()
                    if klm%24 == 0:
                        best_models = best_n_model(track_dict=track_dict, best=20)
                        df = pd.DataFrame.from_dict(track_dict,orient='index')
                        file = '../hyper_Burgers/dict_burgers'
                        df.to_csv(file)
                        
                    print("*******************************************************************************\n")
                    print("Trial {} has been completed".format(klm))
                    print("Time took to complete the trial is {}".format(end-start))
                    print("Total error in the solution is: {:e} ".format(error_loss(y_test, y_hat)))
                    print("Network trained is: {}".format(layers))
                    klm +=1
                    print("Details are: Optimizer: {0}, Learning Rate: {1}, Depth: {2}, Widht: {3}".format(opt, lr, dep, wid))
                    print("Deleting the current trained model")
                    del shared_model
                    del optimizer
                    del small_dict
                    del loss_model
#                     del loss_function
#                     del optimizer
                    print("\n*******************************************************************************\n")
    return track_dict





In [17]:
train_dataset = data_gen(X_f_train, batch_size=X_f_train.shape[0])
loss_function = tf.keras.losses.MeanSquaredError()
track_dict = param_tuning(optim, lr_rate, depth, width, loss_function, train_dataset)

  0%|          | 0/3 [00:00<?, ?it/s]

*******************************************************************************

Trial 0 has been completed
Time took to complete the trial is 65.50399088859558
Total error in the solution is: 5.384618e-01 
Network trained is: [2, 10, 10, 10, 10, 1]
Details are: Optimizer: Adam, Learning Rate: 0.0009, Depth: 4, Widht: 10
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 1 has been completed
Time took to complete the trial is 66.1519467830658
Total error in the solution is: 4.742121e-01 
Network trained is: [2, 20, 20, 20, 20, 1]
Details are: Optimizer: Adam, Learning Rate: 0.0009, Depth: 4, Widht: 20
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 2 has been completed
Time took to 

*******************************************************************************

Trial 17 has been completed
Time took to complete the trial is 73.89776825904846
Total error in the solution is: 4.456051e-01 
Network trained is: [2, 30, 30, 30, 30, 30, 30, 1]
Details are: Optimizer: Adam, Learning Rate: 0.0397, Depth: 6, Widht: 30
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 18 has been completed
Time took to complete the trial is 84.36474013328552
Total error in the solution is: 4.176916e-01 
Network trained is: [2, 10, 10, 10, 10, 10, 10, 10, 10, 1]
Details are: Optimizer: Adam, Learning Rate: 0.0397, Depth: 8, Widht: 10
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 19 has 

*******************************************************************************

Trial 36 has been completed
Time took to complete the trial is 63.7361478805542
Total error in the solution is: 4.569158e-01 
Network trained is: [2, 10, 10, 10, 10, 1]
Details are: Optimizer: Adam, Learning Rate: 0.0248, Depth: 4, Widht: 10
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 37 has been completed
Time took to complete the trial is 63.73188805580139
Total error in the solution is: 4.486128e-01 
Network trained is: [2, 20, 20, 20, 20, 1]
Details are: Optimizer: Adam, Learning Rate: 0.0248, Depth: 4, Widht: 20
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 38 has been completed
Time took 

 33%|███▎      | 1/3 [1:03:32<2:07:04, 3812.34s/it]

...

*******************************************************************************

Trial 95 has been completed
Time took to complete the trial is 111.12175941467285
Total error in the solution is: 1.242193e+00 
Network trained is: [2, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 1]
Details are: Optimizer: RMSprop, Learning Rate: 0.0248, Depth: 10, Widht: 30
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 96 has been completed
Time took to complete the trial is 63.416512966156006
Total error in the solution is: 8.756199e-01 
Network trained is: [2, 10, 10, 10, 10, 1]
Details are: Optimizer: SGD, Learning Rate: 0.0009, Depth: 4, Widht: 10
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 97

*******************************************************************************

Trial 114 has been completed
Time took to complete the trial is 83.31152939796448
Total error in the solution is: 5.801422e-01 
Network trained is: [2, 10, 10, 10, 10, 10, 10, 10, 10, 1]
Details are: Optimizer: SGD, Learning Rate: 0.0397, Depth: 8, Widht: 10
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 115 has been completed
Time took to complete the trial is 83.24454593658447
Total error in the solution is: 5.503624e-01 
Network trained is: [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]
Details are: Optimizer: SGD, Learning Rate: 0.0397, Depth: 8, Widht: 20
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial

*******************************************************************************

Trial 132 has been completed
Time took to complete the trial is 62.49234056472778
Total error in the solution is: 5.956878e-01 
Network trained is: [2, 10, 10, 10, 10, 1]
Details are: Optimizer: SGD, Learning Rate: 0.0248, Depth: 4, Widht: 10
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 133 has been completed
Time took to complete the trial is 66.40578889846802
Total error in the solution is: 6.002579e-01 
Network trained is: [2, 20, 20, 20, 20, 1]
Details are: Optimizer: SGD, Learning Rate: 0.0248, Depth: 4, Widht: 20
Deleting the current trained model

*******************************************************************************

*******************************************************************************

Trial 134 has been completed
Time too

100%|██████████| 3/3 [3:18:15<00:00, 3965.16s/it]  

*******************************************************************************

Trial 143 has been completed
Time took to complete the trial is 92.31963396072388
Total error in the solution is: 5.555078e-01 
Network trained is: [2, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 1]
Details are: Optimizer: SGD, Learning Rate: 0.0248, Depth: 10, Widht: 30
Deleting the current trained model

*******************************************************************************



In [31]:
best_models = best_n_model(track_dict=track_dict, best=20)
pd.DataFrame.from_dict(best_models,orient='index')


,optimizer,learning_rate,number of layer,number of neurons,loss model,error
30,Adam,0.0098,8,10,"[0.21567384619265795, 0.24173839343711734, 0.2...",0.095725
28,Adam,0.0098,6,20,"[0.669098973274231, 1.0847085118293762, 0.4912...",0.170551
27,Adam,0.0098,6,10,"[0.35738835483789444, 0.25885569863021374, 0.2...",0.176880
12,Adam,0.0397,4,10,"[0.5199848860502243, 0.38453294336795807, 0.52...",0.220782
33,Adam,0.0098,10,10,"[0.4157719947397709, 0.26989802275784314, 0.22...",0.291373
11,Adam,0.0009,10,30,"[0.4858335256576538, 0.30999305471777916, 0.28...",0.302693
29,Adam,0.0098,6,30,"[0.6160300597548485, 3.6552160382270813, 0.780...",0.303956
39,Adam,0.0248,6,10,"[0.35738835483789444, 0.49027590453624725, 0.2...",0.314746
10,Adam,0.0009,10,20,"[1.0859741270542145, 0.6248389929533005, 0.351...",0.316655
6,Adam,0.0009,8,10,"[0.21567384619265795, 0.21170908771455288, 0.2...",0.321154


In [32]:
df_best = pd.DataFrame.from_dict(best_models,orient='index')
df_track = pd.DataFrame.from_dict(track_dict,orient='index')

In [33]:
# os.mkdir('../hyper_Burgers')

In [34]:
file = '../hyper_Burgers/'
df_best.to_csv(file + 'best_dict')
df_track.to_csv(file + 'track_dict')

In [35]:
pd.read_csv(file+ 'best_dict')

,Unnamed: 0,optimizer,learning_rate,number of layer,number of neurons,loss model,error
0,30,Adam,0.0098,8,10,"[0.21567384619265795, 0.24173839343711734, 0.2...",0.095725
1,28,Adam,0.0098,6,20,"[0.669098973274231, 1.0847085118293762, 0.4912...",0.170551
2,27,Adam,0.0098,6,10,"[0.35738835483789444, 0.25885569863021374, 0.2...",0.176880
3,12,Adam,0.0397,4,10,"[0.5199848860502243, 0.38453294336795807, 0.52...",0.220782
4,33,Adam,0.0098,10,10,"[0.4157719947397709, 0.26989802275784314, 0.22...",0.291373
5,11,Adam,0.0009,10,30,"[0.4858335256576538, 0.30999305471777916, 0.28...",0.302693
6,29,Adam,0.0098,6,30,"[0.6160300597548485, 3.6552160382270813, 0.780...",0.303956
7,39,Adam,0.0248,6,10,"[0.35738835483789444, 0.49027590453624725, 0.2...",0.314746
8,10,Adam,0.0009,10,20,"[1.0859741270542145, 0.6248389929533005, 0.351...",0.316655
9,6,Adam,0.0009,8,10,"[0.21567384619265795, 0.21170908771455288, 0.2...",0.321154


In [36]:
pd.read_csv(file+ 'track_dict')

,Unnamed: 0,optimizer,learning_rate,number of layer,number of neurons,loss model,error
0,0,Adam,0.0009,4,10,"[0.5199848860502243, 0.49136319756507874, 0.46...",0.538462
1,1,Adam,0.0009,4,20,"[0.24911283422261477, 0.22555065201595426, 0.2...",0.474212
2,2,Adam,0.0009,4,30,"[0.8805493414402008, 0.6395775079727173, 0.476...",0.488953
3,3,Adam,0.0009,6,10,"[0.35738835483789444, 0.33319417387247086, 0.3...",0.424411
4,4,Adam,0.0009,6,20,"[0.669098973274231, 0.46902719140052795, 0.329...",0.460009
...,...,...,...,...,...,...,...
139,139,SGD,0.0248,8,20,"[0.7193073034286499, 1.0514986217021942, 0.674...",0.569289
140,140,SGD,0.0248,8,30,"[0.7023464441299438, 1.6923798620700836, 2.321...",0.552254
141,141,SGD,0.0248,10,10,"[0.4157719947397709, 0.35198673978447914, 0.31...",0.591281
142,142,SGD,0.0248,10,20,"[1.0859741270542145, 2.017742335796356, 0.3668...",0.552593
